# Setup and Load Data

In [1]:
import tensorflow as tf
import os

In [2]:
#To avoid out of memory errors by setting GPU memory consumption growth
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

# Removing dodgy images

In [3]:
import cv2
import imghdr

In [4]:
data_dir='data'

In [5]:
image_exts=['jpeg','jpg','bmp','png']

In [6]:
os.listdir(data_dir)

['Happy', 'Sad']

In [7]:
# Returns every image in the files
os.listdir(os.path.join(data_dir,'Happy'))

['05-12-21-happy-people.jpg',
 '10-Habits-of-Happy-People-Seniors-Today.jpg',
 '1000_F_42220133_toAU6USGY9jVr2XJFLssfG00cSYIJ173.jpg',
 '105846315-makeit_04082019_bajuelos_jon_haidt_happiness_v2.00_03_34_04.still001.jpg',
 '154006829.jpg',
 '170404-happy-workers-feature.jpg',
 '1920px-face-smile.svg_.png',
 '192px.svg',
 '1HEoLBLidT2u4mhJ0oiDgig.png',
 '1rOrTgNQABn0P7dy3IrNDjg.jpeg',
 '220px-Happy_People_A_Year_in_the_Taiga_poster.jpg',
 '2560px-Happy_People.jpg',
 '35438_hd.jpg',
 '56f455011e0000b300705475.jpeg',
 '61vqdCPeKNL.jpg',
 '7-Habits-of-Happy-People-image.jpeg',
 '7-principles-of-successful-and-happy-people.png',
 '71hBPTAhIXL.jpg',
 '835405.jpg',
 '960x0.jpg',
 'AOh-ky2-dvq2OSN9SSeqKr6rbNn_OxX7N18ZcDSkffPas64-c-mo.jpg',
 'A_Sep20_14_1189155141.jpg',
 'Canada-happy-people.jpg',
 'cess-group-positive-friends-celebrate-victory-achievement-together_1016-9922.jpg',
 'compassion-900x387.jpg',
 'Dollarphotoclub_76084977-1.jpg',
 'e162ccda8ce0f197f8863f327add9233.jpg',
 'FitnessMar